In [1]:
import os
os.environ["OPENAI_API_KEY"]=" insert your api key "

In [2]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms.openai import OpenAI
from llama_index.agent.openai import OpenAIAgent
from llama_index.core.tools.tool_spec.base import BaseToolSpec
from duckduckgo_search import DDGS

# === Tools ===

class MathTool(BaseToolSpec):
    spec_functions = ["add", "sub", "mul", "div"]
    def add(self, a: int, b: int) -> int: return a + b
    def sub(self, a: int, b: int) -> int: return a - b
    def mul(self, a: int, b: int) -> int: return a * b
    def div(self, a: int, b: int) -> float: return a / b

f_path = "D:/RAMANASOFT/telugumovies_dataset (2).csv"
reader = SimpleDirectoryReader(input_files=[f_path])
pdf_documents = reader.load_data()
index = VectorStoreIndex.from_documents(pdf_documents)
query_engine = index.as_query_engine()

class DocumentTool(BaseToolSpec):
    spec_functions = ["ask_doc"]
    def ask_doc(self, question: str) -> str:
        return str(query_engine.query(question))

class DuckDuckGoTool(BaseToolSpec):
    spec_functions = ["search_web"]
    def search_web(self, query: str) -> str:
        results = []
        with DDGS() as ddgs:
            for r in ddgs.text(query, max_results=3):
                results.append(f"{r['title']}\n{r['href']}\n{r['body']}\n")
        return "\n---\n".join(results)

# === LLM ===
llm = OpenAI(model="gpt-3.5-turbo", temperature=0)

# === Agents ===
agent_math = OpenAIAgent.from_tools(
    tools=MathTool().to_tool_list(),
    llm=llm,
    verbose=True,
)

agent_doc = OpenAIAgent.from_tools(
    tools=DocumentTool().to_tool_list(),
    llm=llm,
    verbose=True,
)

agent_duck = OpenAIAgent.from_tools(
    tools=DuckDuckGoTool().to_tool_list(),
    llm=llm,
    verbose=True,
)

# === Team Class ===
class Team:
    def __init__(self, agents):
        self.agents = agents

    def run(self, query: str) -> str:
        query_lower = query.lower()
        # Simple routing logic
        if any(word in query_lower for word in ["add", "subtract", "multiply", "divide", "math", "calculate"]):
            return self.agents["math"].chat(query)
        elif any(word in query_lower for word in ["document", "pdf", "alice", "wonderland", "read", "text"]):
            return self.agents["doc"].chat(query)
        else:
            return self.agents["duck"].chat(query)

# === Create team with agents mapped by name ===
team = Team({
    "math": agent_math,
    "doc": agent_doc,
    "duck": agent_duck,
})

# === Main Loop ===
if __name__ == "__main__":
    while True:
        query = input("Ask something (type 'exit' to quit): ")
        if query.lower() in ["exit", "quit"]:
            break
        response = team.run(query)
        print(f"\nResponse:\n{response}")


D:\ju\Lib\site-packages\llama_index\agent\openai\base.py:135: DeprecationWarning: Call to deprecated class AgentRunner. (AgentRunner has been deprecated and is not maintained.

This implementation will be removed in a v0.13.0.

See the docs for more information on updated agent usage: https://docs.llamaindex.ai/en/stable/understanding/agent/)
  return cls(
D:\ju\Lib\site-packages\llama_index\agent\openai\base.py:135: DeprecationWarning: Call to deprecated class AgentRunner. (AgentRunner has been deprecated and is not maintained.

This implementation will be removed in a v0.13.0.

See the docs for more information on updated agent usage: https://docs.llamaindex.ai/en/stable/understanding/agent/)
  return cls(
D:\ju\Lib\site-packages\llama_index\agent\openai\base.py:135: DeprecationWarning: Call to deprecated class AgentRunner. (AgentRunner has been deprecated and is not maintained.

This implementation will be removed in a v0.13.0.

See the docs for more information on updated agent usa

Ask something (type 'exit' to quit):  give me list of movies


Added user message to memory: give me list of movies
=== Calling Function ===
Calling function: search_web with args: {"query":"list of movies"}
Got output: Error: https://html.duckduckgo.com/html RuntimeError: error sending request for url (https://html.duckduckgo.com/html): operation timed out

Caused by:
    operation timed out

=== Calling Function ===
Calling function: search_web with args: {"query": "popular movies"}
Got output: Error: https://lite.duckduckgo.com/lite/ RuntimeError: error sending request for url (https://lite.duckduckgo.com/lite/): operation timed out

Caused by:
    operation timed out

=== Calling Function ===
Calling function: search_web with args: {"query": "upcoming movies"}
Got output: Error: https://html.duckduckgo.com/html RuntimeError: error sending request for url (https://html.duckduckgo.com/html): operation timed out

Caused by:
    operation timed out

=== Calling Function ===
Calling function: search_web with args: {"query": "top movies 2021"}
Got o

Ask something (type 'exit' to quit):  exit
